# Model Selection

In this notebook we will test different models to find the one that gives us the best results.

In [2]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

def f1_scores(Z, y_test):
    averages = ['macro', 'micro', 'weighted']
    for avg in averages:
        score = f1_score(Z, y_test, average=avg)
        print("f1 score ({}): {}".format(avg, score))

def test_model(X, y, model_name, model):
    print("MODEL: {}".format(model_name))
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    model.fit(X_train, y_train)
    Z = model.predict(X_test)
    f1_scores(Z, y_test)
    
def testModelKFold(X, y, model_name, model, k):
    print("CROSS VALIDATION FOR: {}".format(model_name))
    skf = StratifiedKFold(n_splits=5)
    skf.get_n_splits(X, y)
    f1_scores = []
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model.fit(X_train, y_train)
        Z = model.predict(X_test)
        f1_scores.append(f1_score(Z, y_test))
    print("f1 scores: {}".format(f1_scores))
    print("MEAN: {}".format(np.mean(f1_scores)))

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

models = {
    "Logistic Regression": LogisticRegression(),
    "SVC": SVC(),
    "Random Forest": RandomForestClassifier(n_estimators=1000),
    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors = 3),
    "GaussianNB": GaussianNB(),
    "Perceptron": Perceptron(),
    "SGDClassifier": SGDClassifier(),
    "Decision Tree": DecisionTreeClassifier()   
}

dataset_path = "data.csv"
data = pd.read_csv(dataset_path, sep=";")
X = data.drop("diagnosis", axis=1).values
y = data["diagnosis"]

for model in models:
    test_model(X, y, model, models[model])
    testModelKFold(X, y, model, models[model], k=5)
    print('')

MODEL: Logistic Regression
f1 score (macro): 0.7766676048409795
f1 score (micro): 0.7958412098298676
f1 score (weighted): 0.7950990057657817
CROSS VALIDATION FOR: Logistic Regression
f1 scores: [0.8554913294797688, 0.8082595870206489, 0.5481727574750831, 0.7988668555240793, 0.4453441295546559]
MEAN: 0.6912269318108473

MODEL: SVC
f1 score (macro): 0.8477229620224214
f1 score (micro): 0.8620037807183365
f1 score (weighted): 0.8635023851493893
CROSS VALIDATION FOR: SVC
f1 scores: [0.8913043478260869, 0.8385269121813033, 0.7223382045929019, 0.8825214899713467, 0.5522388059701493]
MEAN: 0.7773859521083576

MODEL: Random Forest
f1 score (macro): 0.8732767680514343
f1 score (micro): 0.8827977315689982
f1 score (weighted): 0.8830603788384482
CROSS VALIDATION FOR: Random Forest
f1 scores: [0.9703504043126684, 0.8538011695906433, 0.7098039215686275, 0.8587570621468926, 0.5413533834586466]
MEAN: 0.7868131882154957

MODEL: K-Nearest Neighbors
f1 score (macro): 0.8063196434256605
f1 score (micro):

We can check the confusion matrix for the Random Forest model.

In [5]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

model = RandomForestClassifier(n_estimators=1000)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model.fit(X_train, y_train)
Z = model.predict(X_test)

print("Confusion Matrix")
print(confusion_matrix(y_test, Z))

print("\nClassfication Report")
print(classification_report(y_test, Z))

Confusion Matrix
[[320  32]
 [ 32 145]]

Classfication Report
             precision    recall  f1-score   support

          0       0.91      0.91      0.91       352
          1       0.82      0.82      0.82       177

avg / total       0.88      0.88      0.88       529



## Save model

In [10]:
import dill as pickle

def dump_model(model, path):
    with open(path, 'wb') as file:
        pickle.dump(model, file)
        
# Define path to save the model file
filename = "model_v1.pk"
dir_path = "./"

# Dump model to file
dump_model(model, dir_path + filename)